# 1. Data Wrangling

Aqui será realizada a ingestão dos dados brutos e alguns tratamentos de limpeza dos dados, tais como a verificação de dados duplicados, a correção de inconsistências nos campos das datas, entre outras operações.

Contêm também o merge entre as bases e por fim é armazenada a nova base para a etapa de Feature Engineering.

In [1]:
# Bibliotecas
import pandas as pd
import numpy as np

****
### 1.1. Base cadastral

Variáveis:
- ID_CLIENTE: Identificador único do cliente.
- DATA_CADASTRO: Data da realização do cadastro no sistema.
- DDD: Número do DDD do telefone do cliente.
- FLAG_PF: Indica se o cliente é uma pessoa física ('X') ou jurídica ('NaN').
- SEGMENTO_INDUSTRIAL: Indica a qual segmento da indústria pertence o cliente.
- DOMINIO_EMAIL: Indica o domínio (ou provedor) do email utilizado para o cadastro.
- PORTE: Indica o porte (tamanho) da empresa.
- CEP_2_DIG: Indica os dois primeiros números do CEP do endereço cadastrado.

In [2]:
df_base_cadastral = pd.read_csv('../data/1_raw/base_cadastral.csv')
df_base_cadastral.head(5)

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,8274986328479596038,2017-01-25,31,NaN,Comércio,YAHOO,MEDIO,77
2,345447888460137901,2000-08-15,75,NaN,Serviços,HOTMAIL,PEQUENO,48
3,1003144834589372198,2017-08-06,49,NaN,Serviços,OUTLOOK,PEQUENO,89
4,324916756972236008,2011-02-14,88,NaN,Serviços,GMAIL,GRANDE,62


In [3]:
df_base_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_CLIENTE           1315 non-null   int64 
 1   DATA_CADASTRO        1315 non-null   object
 2   DDD                  1078 non-null   object
 3   FLAG_PF              66 non-null     object
 4   SEGMENTO_INDUSTRIAL  1232 non-null   object
 5   DOMINIO_EMAIL        1285 non-null   object
 6   PORTE                1274 non-null   object
 7   CEP_2_DIG            1315 non-null   object
dtypes: int64(1), object(7)
memory usage: 82.3+ KB


In [4]:
# Filtrando para um cliente qualquer como exemplo
df_base_cadastral[df_base_cadastral['ID_CLIENTE'] == 1661240395903230676]

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65


In [5]:
print(df_base_cadastral['SEGMENTO_INDUSTRIAL'].unique())
print(df_base_cadastral['DOMINIO_EMAIL'].unique())
print(df_base_cadastral['PORTE'].unique())

['Serviços' 'Comércio' 'Indústria' nan]
['YAHOO' 'HOTMAIL' 'OUTLOOK' 'GMAIL' 'BOL' 'AOL' nan]
['PEQUENO' 'MEDIO' 'GRANDE' nan]


In [6]:
# Verifica se há clientes duplicados. Se soma=0, não há.
print(f"Quantidade de clientes duplicados: ", df_base_cadastral.duplicated(subset=['ID_CLIENTE']).sum())

Quantidade de clientes duplicados:  0


#### 1.1.1. Tratamentos na Base Cadastral

- FLAG_PF:
    - serão atribuídos labels específicos para ambos casos, 'PF' e 'PJ'.
- DDD:
    - Variável importante que denota localização geográfica;
    - Só há DDD com 2 dígitos. Onde ocorre 1 dígito ou "(" será substituído por NaN;
    - Como o primeiro dígito sozinho já carrega informação de localidade, nos casos que há "(x", onde __x__ é um número, esse número será considerado o primeiro dígito do DDD, imaginando que tenha sido recebido de uma inputação onde o DDD tenha sido deslocado uma posição pela inclusão do parenteses.
    - casos NaN será atribuído '0'
- CEP_2_DIG:
    - Há 1 registro com o valor 'na' (str). Será substituído por NaN.
    - casos NaN será atribuído '0'

In [7]:
print("FLAG_PF:")
print("Anterior: ", df_base_cadastral['FLAG_PF'].unique())
df_base_cadastral['FLAG_PF'] = df_base_cadastral['FLAG_PF'].replace(np.nan, 'PJ').replace('X', 'PF')
print("Novo:     ", df_base_cadastral['FLAG_PF'].unique())

print("\nDDD:")
print("Anterior (len): ", len(df_base_cadastral['DDD'].unique()))
df_base_cadastral['DDD_1'] = df_base_cadastral['DDD'].str[0]
df_base_cadastral['DDD_2'] = df_base_cadastral['DDD'].str[1]
df_base_cadastral['DDD'] = [ np.nan if x == '(' else y for x, y in zip(df_base_cadastral['DDD_1'], df_base_cadastral['DDD']) ]
df_base_cadastral['DDD_1'] = [row['DDD_2'] if row['DDD_1'] == '(' else row['DDD_1'] for _, row in df_base_cadastral.iterrows()]
df_base_cadastral = df_base_cadastral.drop(columns='DDD_2')

print("Novo     (len): ", len(df_base_cadastral['DDD'].unique()))

print("\nCEP_2_DIG:")
df_base_cadastral['CEP_2_DIG'] = df_base_cadastral['CEP_2_DIG'].replace('na', np.nan)
print("len: \t", len(df_base_cadastral['CEP_2_DIG'].unique()))

# Colunas que apesar de serem inteiros, denotam categorias
df_base_cadastral['DDD'] = df_base_cadastral['DDD'].fillna(0).astype(int).astype(str)
df_base_cadastral['CEP_2_DIG'] = df_base_cadastral['CEP_2_DIG'].fillna(0).astype(int).astype(str)
df_base_cadastral['DDD_1'] = df_base_cadastral['DDD_1'].fillna(0).astype(int).astype(str)


df_base_cadastral

FLAG_PF:
Anterior:  [nan 'X']
Novo:      ['PJ' 'PF']

DDD:
Anterior (len):  80
Novo     (len):  70

CEP_2_DIG:
len: 	 90


,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1
0,1661240395903230676,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
1,8274986328479596038,2017-01-25,31,PJ,Comércio,YAHOO,MEDIO,77,3
2,345447888460137901,2000-08-15,75,PJ,Serviços,HOTMAIL,PEQUENO,48,7
3,1003144834589372198,2017-08-06,49,PJ,Serviços,OUTLOOK,PEQUENO,89,4
4,324916756972236008,2011-02-14,88,PJ,Serviços,GMAIL,GRANDE,62,8
...,...,...,...,...,...,...,...,...,...
1310,3431426889924624821,2020-08-13,92,PJ,Serviços,HOTMAIL,MEDIO,69,9
1311,5288503299611498087,2020-11-03,0,PJ,Comércio,YAHOO,PEQUENO,13,0
1312,957773253650890560,2021-07-05,0,PJ,Comércio,GMAIL,MEDIO,20,0
1313,6094038865287329652,2021-07-05,0,PJ,Serviços,GMAIL,GRANDE,48,0


****
### 1.2. Base Info

Variáveis:
- ID_CLIENTE: Identificador único do cliente.
- SAFRA_REF: Mês de referência da amostra.
- RENDA_MES_ANTERIOR: Renda ou faturamento declarado pelo cliente no fim do mês anterior.
- NO_FUNCIONARIOS: Número de funcionários reportado pelo cliente no fim do mês anterior.

In [8]:
df_base_info = pd.read_csv('../data/1_raw/base_info.csv')
df_base_info

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,1661240395903230676,2018-09,16913.0,92.0
1,8274986328479596038,2018-09,106430.0,141.0
2,345447888460137901,2018-09,707439.0,99.0
3,1003144834589372198,2018-09,239659.0,96.0
4,324916756972236008,2018-09,203123.0,103.0
...,...,...,...,...
24396,705648002974742140,2021-12,278663.0,105.0
24397,4993499380140734678,2021-12,156968.0,140.0
24398,4614484019183480654,2021-12,292698.0,121.0
24399,1299146298565441811,2021-12,106180.0,121.0


In [9]:
df_base_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24401 entries, 0 to 24400
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_CLIENTE          24401 non-null  int64  
 1   SAFRA_REF           24401 non-null  object 
 2   RENDA_MES_ANTERIOR  24401 non-null  float64
 3   NO_FUNCIONARIOS     24401 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 762.7+ KB


In [10]:
# Filtrando para um cliente qualquer como exemplo para visualizar a temporalidade da base
df_base_info[df_base_info['ID_CLIENTE'] == 1661240395903230676]

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,1661240395903230676,2018-09,16913.0,92.0
569,1661240395903230676,2018-10,236447.0,93.0
1166,1661240395903230676,2018-11,107840.0,91.0
1890,1661240395903230676,2018-12,164029.0,87.0
2435,1661240395903230676,2019-01,128674.0,96.0
3096,1661240395903230676,2019-02,130030.0,93.0
3726,1661240395903230676,2019-03,385018.0,98.0
4281,1661240395903230676,2019-04,287079.0,100.0
4687,1661240395903230676,2019-05,122272.0,101.0
5081,1661240395903230676,2019-06,321136.0,93.0


In [11]:
# Cada cliente deve aparecer apenas uma vez em cada mês de referência
print(f"Quantidade de clientes que aparecem mais de uma vez no mês de referência (SAFRA_REF): ", df_base_info.duplicated(subset=['ID_CLIENTE', 'SAFRA_REF']).sum())
df_base_info.groupby(['ID_CLIENTE', 'SAFRA_REF']).count()

Quantidade de clientes que aparecem mais de uma vez no mês de referência (SAFRA_REF):  0


RENDA_MES_ANTERIOR  NO_FUNCIONARIOS
ID_CLIENTE          SAFRA_REF                                     
8784237149961904    2018-09                     1                1
                    2018-10                     1                1
                    2018-11                     1                1
                    2018-12                     1                1
                    2019-01                     1                1
...                                           ...              ...
9206030810342980458 2021-08                     1                1
                    2021-09                     1                1
                    2021-10                     1                1
                    2021-11                     1                1
                    2021-12                     1                1

[24401 rows x 2 columns]

****
### 1.3. Base de pagamentos

Variáveis:
- ID_CLIENTE: Identificador único do cliente.
- SAFRA_REF: Mês de referência da amostra.
- DATA_EMISSAO_DOCUMENTO: Data da emissão da nota de crédito.
- DATA_VENCIMENTO: Data limite para pagamento do empréstimo.
- VALOR_A_PAGAR: Valor da nota de crédito.
- TAXA: Taxa de juros cobrada no empréstimo.
- DATA_PAGAMENTO: Data em que o cliente realizou o pagamento da nota (disponível apenas na base de desenvolvimento).

In [12]:
df_base_pagamentos_dev = pd.read_csv('../data/1_raw/base_pagamentos_desenvolvimento.csv')
df_base_pagamentos_dev.head(5)

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99


In [13]:
df_base_pagamentos_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              77414 non-null  int64  
 1   SAFRA_REF               77414 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object 
 3   DATA_PAGAMENTO          77414 non-null  object 
 4   DATA_VENCIMENTO         77414 non-null  object 
 5   VALOR_A_PAGAR           77414 non-null  float64
 6   TAXA                    77414 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.1+ MB


In [14]:
# Filtrando para o mesmo cliente qualquer para visualizar melhor essa base
df_base_pagamentos_dev[df_base_pagamentos_dev['ID_CLIENTE'] == 1661240395903230676]

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99
...,...,...,...,...,...,...,...
75035,1661240395903230676,2021-05,2021-05-28,2021-06-17,2021-06-17,42502.86,4.99
75036,1661240395903230676,2021-05,2021-05-30,2021-06-21,2021-06-21,42502.86,4.99
75037,1661240395903230676,2021-06,2021-06-04,2021-06-24,2021-06-24,21501.70,4.99
77019,1661240395903230676,2021-06,2021-06-20,2021-07-12,2021-07-12,21572.90,5.99


### 1.3.1. Tratamentos na Base de Pagamentos

#### Definição de INADIMPLENCIA

Verifica prazo entre data de vencimento e data de pagamento, e define caso de inadimplência, quando o pagamento foi realizado com 5 dias ou mais após o vencimento.

In [15]:
# Converte colunas com data para detetime
df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO'])
df_base_pagamentos_dev['DATA_PAGAMENTO'] = pd.to_datetime(df_base_pagamentos_dev['DATA_PAGAMENTO'])
df_base_pagamentos_dev['DATA_VENCIMENTO'] = pd.to_datetime(df_base_pagamentos_dev['DATA_VENCIMENTO'])

# Gera variável DIAS_ATRASO, que contém os dias corridos entre a data de vencimento e a data de pagamento
df_base_pagamentos_dev['DIAS_ATRASO'] = (df_base_pagamentos_dev['DATA_PAGAMENTO'] - df_base_pagamentos_dev['DATA_VENCIMENTO']).dt.days

# Caso o DIAS_ATRASO seja maior ou igual a 5 dias, será considerado um caso de inadimplência, sinalizado como True ou False na variável INADIMPLENTE
df_base_pagamentos_dev['INADIMPLENTE'] = df_base_pagamentos_dev.apply(lambda x: x['DIAS_ATRASO'] >= 5, axis=1)

df_base_pagamentos_dev

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,0,False
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,1,False
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,1,False
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,6,True
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,0,False
...,...,...,...,...,...,...,...,...,...
77409,2951563549197799278,2021-06,2021-06-30,2021-07-16,2021-07-16,89980.00,5.99,0,False
77410,5220206408301580591,2021-06,2021-06-30,2021-08-16,2021-08-16,42239.00,5.99,0,False
77411,5860276371789140450,2021-06,2021-06-30,2021-07-16,2021-07-16,20921.50,5.99,0,False
77412,2814790209436551216,2021-06,2021-06-30,2021-07-16,2021-07-16,90231.05,6.99,0,False


#### Inconsistência entre data de pagamento e data de emissão do documento

Será considerada uma inconsistência quando ocorrer da data de pagamento ser anterior à data de emissão do documento.

Nesses casos, optei por substituir a data de pagamento pela data de emissão.

In [16]:
# Gera variável DIAS_EMISSAO_PAGAMENTO, que contém os dias corridos entre a data de emissão e a data de pagamento
df_base_pagamentos_dev['DIAS_EMISSAO_PAGAMENTO'] = (df_base_pagamentos_dev['DATA_PAGAMENTO'] - df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO']).dt.days

# Caso o DIAS_EMISSAO_PAGAMENTO seja menor que 0, será considerado um caso de inconsistencia, pois o documento teria sido pago antes da emissão, sinalizado como True ou False na variável INCONSISTENCIA
df_base_pagamentos_dev['INCONSISTENCIA'] = df_base_pagamentos_dev.apply(lambda x: x['DIAS_EMISSAO_PAGAMENTO'] < 0, axis=1)

df_inconsistencia = df_base_pagamentos_dev[df_base_pagamentos_dev['INCONSISTENCIA']]

print(f"Há {len(df_inconsistencia)} registros com data de pagamento antes da data de emissão")
df_inconsistencia

Há 26 registros com data de pagamento antes da data de emissão


,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO,INCONSISTENCIA
5481,8198518259946944874,2018-11,2018-11-13,2018-06-19,2017-12-28,255.43,6.99,173,True,-147,True
10305,8733525963966339207,2019-01,2019-01-04,2018-12-27,2019-01-21,15143.60,6.99,-25,False,-8,True
15756,2209806355660046187,2019-03,2019-03-24,2019-02-26,2019-04-09,1356.70,11.99,-42,False,-26,True
17888,6056959220685085102,2019-04,2019-04-18,2019-04-12,2019-05-15,1182.00,5.99,-33,False,-6,True
18188,660382477567765256,2019-05,2019-05-13,2019-04-23,2019-01-23,600000.00,4.99,90,True,-20,True
25889,8047821974672655407,2019-08,2019-08-12,2019-06-11,2019-06-06,500000.00,6.99,5,True,-62,True
32778,8609938720215175911,2019-10,2019-10-29,2019-09-30,2019-11-18,34298.55,5.99,-49,False,-29,True
37827,6693181132429641622,2020-01,2020-01-05,2019-12-26,2019-12-26,88000.00,4.99,0,False,-10,True
38057,1292049196559782171,2020-01,2020-01-17,2020-01-07,2020-02-01,327.60,5.99,-25,False,-10,True
38065,1292049196559782171,2020-02,2020-02-11,2020-01-09,2020-02-26,599.13,5.99,-48,False,-33,True


In [17]:
# Trata essa inconsistencia atribuindo a data de pagamento à data de emissão
df_base_pagamentos_dev.loc[df_base_pagamentos_dev['INCONSISTENCIA'], 'DATA_PAGAMENTO'] = df_base_pagamentos_dev.loc[df_base_pagamentos_dev['INCONSISTENCIA'], 'DATA_EMISSAO_DOCUMENTO']

# Verifica se o problema foi resolvido
df_base_pagamentos_dev['DIAS_EMISSAO_PAGAMENTO'] = (df_base_pagamentos_dev['DATA_PAGAMENTO'] - df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO']).dt.days
df_base_pagamentos_dev['INCONSISTENCIA'] = df_base_pagamentos_dev.apply(lambda x: x['DIAS_EMISSAO_PAGAMENTO'] < 0, axis=1)
df_inconsistencia = df_base_pagamentos_dev[df_base_pagamentos_dev['INCONSISTENCIA']]

print(f"Agora há {len(df_inconsistencia)} registros com data de pagamento antes da data de emissão")
df_inconsistencia

Agora há 0 registros com data de pagamento antes da data de emissão


,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO,INCONSISTENCIA


In [18]:
# Dropa coluna auxiliar
df_base_pagamentos_dev = df_base_pagamentos_dev.drop(columns=['INCONSISTENCIA'])
df_base_pagamentos_dev

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,0,False,20
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,1,False,23
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,1,False,23
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,6,True,42
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,0,False,20
...,...,...,...,...,...,...,...,...,...,...
77409,2951563549197799278,2021-06,2021-06-30,2021-07-16,2021-07-16,89980.00,5.99,0,False,16
77410,5220206408301580591,2021-06,2021-06-30,2021-08-16,2021-08-16,42239.00,5.99,0,False,47
77411,5860276371789140450,2021-06,2021-06-30,2021-07-16,2021-07-16,20921.50,5.99,0,False,16
77412,2814790209436551216,2021-06,2021-06-30,2021-07-16,2021-07-16,90231.05,6.99,0,False,16


#### Inconsistência entre data de vencimento e data de emissão do documento

Será considerada uma inconsistência quando ocorrer da data de vencimento ser anterior à data de emissão do documento.

Nesses casos, optei por substituir a data de vencimento pela data de emissão.

In [19]:
# Gera variável DIAS_EMISSAO_VENCIMENTO, que contém os dias corridos entre a data de emissão e a data de pagamento
df_base_pagamentos_dev['DIAS_EMISSAO_VENCIMENTO'] = (df_base_pagamentos_dev['DATA_VENCIMENTO'] - df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO']).dt.days

# Caso o DIAS_EMISSAO_VENCIMENTO seja menor que 0, será considerado um caso de inconsistencia, pois o documento teria sido pago antes da emissão, sinalizado como True ou False na variável INCONSISTENCIA
df_base_pagamentos_dev['INCONSISTENCIA'] = df_base_pagamentos_dev.apply(lambda x: x['DIAS_EMISSAO_VENCIMENTO'] < 0, axis=1)

df_inconsistencia = df_base_pagamentos_dev[df_base_pagamentos_dev['INCONSISTENCIA']]

print(f"Há {len(df_inconsistencia)} registros com data de vencimento antes da data de emissão")
df_inconsistencia

Há 27 registros com data de vencimento antes da data de emissão


,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO,DIAS_EMISSAO_VENCIMENTO,INCONSISTENCIA
1099,1118514612547157551,2018-09,2018-09-02,2018-09-19,2018-08-27,58434.08,6.99,23,True,17,-6,True
1100,1118514612547157551,2018-09,2018-09-02,2018-09-18,2018-06-20,50000.00,6.99,90,True,16,-74,True
1101,1118514612547157551,2018-09,2018-09-02,2018-09-19,2018-07-25,58434.08,6.99,56,True,17,-39,True
1944,1977574645216036803,2018-09,2018-09-13,2018-09-28,2018-08-31,46674.40,5.99,28,True,15,-13,True
2565,8198518259946944874,2018-10,2018-10-12,2019-05-22,2017-11-27,11878.82,4.99,541,True,222,-319,True
5009,6020593949794792044,2018-11,2018-11-06,2021-03-23,2018-11-05,278657.56,6.99,869,True,868,-1,True
5481,8198518259946944874,2018-11,2018-11-13,2018-11-13,2017-12-28,255.43,6.99,173,True,0,-320,True
8379,8348574220426911086,2019-01,2019-01-03,2019-01-16,2018-12-17,10000.00,6.99,30,True,13,-17,True
16200,4442205629227031623,2019-03,2019-03-24,2019-06-12,2018-12-07,100000.00,5.99,187,True,80,-107,True
16201,4442205629227031623,2019-03,2019-03-24,2019-06-14,2018-12-07,63000.00,5.99,189,True,82,-107,True


In [20]:
# Trata essa inconsistencia atribuindo a data de pagamento à data de emissão
df_base_pagamentos_dev.loc[df_base_pagamentos_dev['INCONSISTENCIA'], 'DATA_VENCIMENTO'] = df_base_pagamentos_dev.loc[df_base_pagamentos_dev['INCONSISTENCIA'], 'DATA_EMISSAO_DOCUMENTO']

# Verifica se o problema foi resolvido
df_base_pagamentos_dev['DIAS_EMISSAO_VENCIMENTO'] = (df_base_pagamentos_dev['DATA_VENCIMENTO'] - df_base_pagamentos_dev['DATA_EMISSAO_DOCUMENTO']).dt.days
df_base_pagamentos_dev['INCONSISTENCIA'] = df_base_pagamentos_dev.apply(lambda x: x['DIAS_EMISSAO_VENCIMENTO'] < 0, axis=1)
df_inconsistencia = df_base_pagamentos_dev[df_base_pagamentos_dev['INCONSISTENCIA']]

print(f"Agora há {len(df_inconsistencia)} registros com data de vencimento antes da data de emissão")
df_inconsistencia

Agora há 0 registros com data de vencimento antes da data de emissão


,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO,DIAS_EMISSAO_VENCIMENTO,INCONSISTENCIA


In [21]:
# Dropa coluna auxiliar
df_base_pagamentos_dev = df_base_pagamentos_dev.drop(columns=['INCONSISTENCIA'])
df_base_pagamentos_dev

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,INADIMPLENTE,DIAS_EMISSAO_PAGAMENTO,DIAS_EMISSAO_VENCIMENTO
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99,0,False,20,20
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99,1,False,23,22
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99,1,False,23,22
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99,6,True,42,36
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99,0,False,20,20
...,...,...,...,...,...,...,...,...,...,...,...
77409,2951563549197799278,2021-06,2021-06-30,2021-07-16,2021-07-16,89980.00,5.99,0,False,16,16
77410,5220206408301580591,2021-06,2021-06-30,2021-08-16,2021-08-16,42239.00,5.99,0,False,47,47
77411,5860276371789140450,2021-06,2021-06-30,2021-07-16,2021-07-16,20921.50,5.99,0,False,16,16
77412,2814790209436551216,2021-06,2021-06-30,2021-07-16,2021-07-16,90231.05,6.99,0,False,16,16


****
### 1.4. Merge das bases

Obs.: Fazendo esse merge com 'inner', removo a parte da Base Info que contêm informações de SAFRA_REF do período de 07-2021 à 12/2021. Quando for fazer a inferência, fazer o merge entre a Base Pagamento Teste e a Base Info e lá esses dados serão considerados.

In [22]:
df_merged = df_base_info.merge(df_base_pagamentos_dev, how='inner', on=['ID_CLIENTE', 'SAFRA_REF'])
df_merged = df_merged.merge(df_base_cadastral, how='inner', on='ID_CLIENTE')

df_merged

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DIAS_EMISSAO_PAGAMENTO,DIAS_EMISSAO_VENCIMENTO,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,23,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,22,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,22,21,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,21,20,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,22,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,43,45,2015-12-13,0,PJ,Comércio,BOL,MEDIO,36,0
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,14,15,2019-07-02,0,PJ,Indústria,BOL,PEQUENO,13,0
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,14,15,2019-07-02,0,PJ,Indústria,BOL,PEQUENO,13,0
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,29,25,2019-12-30,19,PJ,Indústria,OUTLOOK,MEDIO,20,1


****
### 1.5. Tratamentos
#### Tratamento de inconsistência entre as datas de cadastro do cliente e emissão do documento

Após mergear as bases, foi possível notar inconsistência também entre as datas de cadastro do cliente e emissão de documento.

Há casos onde a data de cadastro é posterior à data de emissão do documento.

In [23]:
df = df_merged.copy()

# Converter colunas com data de str para datetime
df['DATA_CADASTRO'] = pd.to_datetime(df['DATA_CADASTRO'])
df['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(df['DATA_EMISSAO_DOCUMENTO'])

# Gera variável TEMPO_VIDA_CLIENTE, que contém os dias corridos entre a data de cadastro e a data de emissão
df['TEMPO_VIDA_CLIENTE'] = (df['DATA_EMISSAO_DOCUMENTO'] - df['DATA_CADASTRO']).dt.days

# Caso o TEMPO_VIDA_CLIENTE seja menor que 0, será considerado um caso de inconsistencia, pois o documento teria sido emitido antes do cadastro do cliente, sinalizado como True ou False na variável INCONSISTENCIA
df['INCONSISTENCIA'] = df.apply(lambda x: x['TEMPO_VIDA_CLIENTE'] < 0, axis=1)
df_inconsistencia = df[df['INCONSISTENCIA']]

print(f"Há {len(df_inconsistencia)} registros com data de cadastro posterior à data de emissão")
df_inconsistencia

Há 18 registros com data de cadastro posterior à data de emissão


,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,INCONSISTENCIA
67043,6632750819478927641,2019-02,408746.0,130.0,2019-02-06,2019-03-14,2019-03-14,1603.82,4.99,0,...,2019-02-18,18,PJ,Indústria,HOTMAIL,GRANDE,16,1,-12,True
69808,8090492548102964782,2019-10,282705.0,111.0,2019-10-11,2019-11-11,2019-11-11,3625.60,5.99,0,...,2019-10-22,11,PJ,Serviços,GMAIL,GRANDE,13,1,-11,True
69886,5043004928991692540,2019-10,90887.0,106.0,2019-10-13,2020-10-29,2019-10-25,8299.78,8.99,370,...,2019-10-24,19,PJ,Comércio,HOTMAIL,MEDIO,13,1,-11,True
69887,5043004928991692540,2019-10,90887.0,106.0,2019-10-13,2020-10-29,2019-10-13,8299.78,8.99,400,...,2019-10-24,19,PJ,Comércio,HOTMAIL,MEDIO,13,1,-11,True
70117,1849408830951333776,2019-11,353416.0,115.0,2019-11-11,2019-11-27,2019-11-27,2365.00,5.99,0,...,2019-11-24,0,PJ,Comércio,HOTMAIL,GRANDE,85,4,-13,True
70118,1849408830951333776,2019-11,353416.0,115.0,2019-11-21,2019-12-16,2019-12-16,3633.43,5.99,0,...,2019-11-24,0,PJ,Comércio,HOTMAIL,GRANDE,85,4,-3,True
70916,3540559576342636557,2020-02,349413.0,124.0,2020-02-11,2020-05-21,2020-04-03,15000.00,8.99,48,...,2020-02-12,0,PJ,Indústria,YAHOO,PEQUENO,21,0,-1,True
71759,5036870533568879731,2020-06,101693.0,128.0,2020-06-11,2020-07-31,2020-07-17,28920.00,8.99,14,...,2020-06-23,0,PJ,Indústria,HOTMAIL,MEDIO,78,6,-12,True
71778,4187591029730357479,2020-06,179943.0,131.0,2020-06-15,2020-06-30,2020-07-07,173538.00,6.99,-7,...,2020-06-18,0,PJ,Indústria,GMAIL,PEQUENO,78,0,-3,True
71882,7038761645302069945,2020-07,113937.0,116.0,2020-07-14,2020-07-29,2020-07-29,132300.00,6.99,0,...,2020-07-21,55,PJ,Comércio,YAHOO,MEDIO,48,5,-7,True


Para esses casos, foi utilizada como data de cadastro a mesma data da emissão do documento.

Optei por fazer essa correção, pois a data de cadastro do cliente poderá ser utilizada como feature do modelo.

In [24]:
# Trata essa inconsistencia atribuindo a data de emissão à data de cadastro
df.loc[df['INCONSISTENCIA'], 'DATA_EMISSAO_DOCUMENTO'] = df.loc[df['INCONSISTENCIA'], 'DATA_CADASTRO']

# Verifica se o problema foi resolvido
df['TEMPO_VIDA_CLIENTE'] = (df['DATA_EMISSAO_DOCUMENTO'] - df['DATA_CADASTRO']).dt.days
df['INCONSISTENCIA'] = df.apply(lambda x: x['TEMPO_VIDA_CLIENTE'] < 0, axis=1)
df_inconsistencia = df[df['INCONSISTENCIA']]

print(f"Agora há {len(df_inconsistencia)} registros com data de pagamento antes da data de emissão")
df_inconsistencia

Agora há 0 registros com data de pagamento antes da data de emissão


,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,INCONSISTENCIA


In [25]:
# Dropa coluna auxiliar
df = df.drop(columns=['INCONSISTENCIA'])
df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DIAS_EMISSAO_VENCIMENTO,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1851
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1858
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,21,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1873
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,20,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1882
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,22,2013-08-22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,45,2015-12-13,0,PJ,Comércio,BOL,MEDIO,36,0,2009
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,15,2019-07-02,0,PJ,Indústria,BOL,PEQUENO,13,0,713
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,15,2019-07-02,0,PJ,Indústria,BOL,PEQUENO,13,0,716
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,25,2019-12-30,19,PJ,Indústria,OUTLOOK,MEDIO,20,1,532


Com essa alteração na data de cadastro de alguns clientes, agora eu posso ter um mesmo cliente com datas de cadastro diferentes.

A solução é buscar nesse dataframe a primeira data de cadastro de cada cliente (data mais antiga) e corrigir para todo o dataframe.

In [26]:
# Busca a data de cadastro mais antiga para cada cliente e atribui em um novo dataframe
df_primeiro_cadastro = df.groupby('ID_CLIENTE')['DATA_CADASTRO'].first().to_frame().reset_index()

# Dropa coluna DATA_CADASTRO do dataframe original para receber a coluna com datas corrigidas
df = df.drop(columns='DATA_CADASTRO')

# Mergeia novo dataframe no anterior, realocando a coluna DATA_CADASTRO corrigida
df = df.merge(df_primeiro_cadastro, on='ID_CLIENTE')

# Gera novamente a variável TEMPO_VIDA_CLIENTE, dessa vez com a DATA_CADASTRO corrigida
df['TEMPO_VIDA_CLIENTE'] = (df['DATA_EMISSAO_DOCUMENTO'] - df['DATA_CADASTRO']).dt.days

df

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,DIAS_ATRASO,...,DIAS_EMISSAO_VENCIMENTO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG,DDD_1,TEMPO_VIDA_CLIENTE,DATA_CADASTRO
0,1661240395903230676,2018-09,16913.0,92.0,2018-09-16,2018-10-09,2018-10-08,22427.25,5.99,1,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1851,2013-08-22
1,1661240395903230676,2018-09,16913.0,92.0,2018-09-23,2018-10-15,2018-10-15,35608.11,5.99,0,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1858,2013-08-22
2,1661240395903230676,2018-10,236447.0,93.0,2018-10-08,2018-10-30,2018-10-29,17988.49,5.99,1,...,21,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1873,2013-08-22
3,1661240395903230676,2018-10,236447.0,93.0,2018-10-17,2018-11-07,2018-11-06,41998.20,6.99,1,...,20,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1882,2013-08-22
4,1661240395903230676,2018-10,236447.0,93.0,2018-10-21,2018-11-12,2018-11-12,35514.41,6.99,0,...,22,99,PJ,Serviços,YAHOO,PEQUENO,65,9,1886,2013-08-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73472,3148228961195376491,2021-06,343507.0,134.0,2021-06-13,2021-07-26,2021-07-28,17302.05,4.99,-2,...,45,0,PJ,Comércio,BOL,MEDIO,36,0,2009,2015-12-13
73473,2439954154030259398,2021-06,392975.0,121.0,2021-06-14,2021-06-28,2021-06-29,19799.30,6.99,-1,...,15,0,PJ,Indústria,BOL,PEQUENO,13,0,713,2019-07-02
73474,2439954154030259398,2021-06,392975.0,121.0,2021-06-17,2021-07-01,2021-07-02,19913.15,11.99,-1,...,15,0,PJ,Indústria,BOL,PEQUENO,13,0,716,2019-07-02
73475,4799794458216343260,2021-06,70449.0,141.0,2021-06-14,2021-07-13,2021-07-09,2806.09,6.99,4,...,25,19,PJ,Indústria,OUTLOOK,MEDIO,20,1,532,2019-12-30


****
### 1.6. Persistência dos dados

In [27]:
# Salva dataframe final para a etapa de Feature Engineering
df.to_csv('../data/2_intermediate/1_df_merged.csv', index=False)

****
## Resumo

- Realizada a importação das bases
- Análise superficial para primeiro contato com esses dados
- Verificação de duplicatas
- Correção de erro de inputação
- Correção de inconsistência de datas
    - data de pagamento < data de emissão
    - data de vencimento < data de emissão
    - data de emissão < data de cadastro
- Mantive variaveis criadas nas correções que poderão ser utilizadas como novas features do modelo
- Merge das bases
- Criação do target:
    - INADIMPLENTE: Se atrasou o pagamento prazo igual ou superior à 5 dias após o vencimento
- Criação de novas features:
    - TEMPO_VIDA_CLIENTE: Tempo de vida do cliente até a emissão do respectivo empréstimo

    